<a href="https://colab.research.google.com/github/tohe0133/ATHENA/blob/main/Test_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install streamlit
! pip install -U accelerate
! pip install -U transformers
!pip install transformers[torch]
!pip install accelerate -U
# A dependency of the preprocessing for BERT inputs
!pip install -U "tensorflow-text==2.14.*"
!pip install "tf-models-official==2.14.*"
!pip install datasets
!pip install transformers
!pip install mlflow
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00
     

In [5]:
#import os
#import shutil

#import streamlit as st
#import numpy as np
import pandas as pd

#from transformers import BertTokenizer
#from datasets import Dataset
#import torch
#import tensorflow as tf
#import tensorflow_hub as hub
#import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer
#from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
#from transformers import AutoModelForSequenceClassification
from transformers import pipeline #TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, pipeline, Trainer, TrainingArguments, DataCollatorWithPadding

#import matplotlib.pyplot as plt

#tf.get_logger().setLevel('ERROR')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
root_dir = '/content/gdrive/MyDrive/'
base_dir = root_dir + 'FIMI_dataset/'
save_dir = root_dir + 'FIMI_Project_Datasets/'
output_dir = base_dir +"fake-news-classifier/"

Mounted at /content/gdrive/


In [6]:
test_df = pd.read_csv(save_dir + 'dataset_test.csv')
test_df = test_df.drop('Unnamed: 0', axis=1)
#test_df = test_df.iloc[:100,:]

tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [7]:
len(test_df)

13656

In [8]:
idx = 13652
print(test_df['label'][idx])
test_df['text'][idx]

true


'The announcement by House Speaker John Boehner that he is retiring at the end of October stunned Washington where life is all about grabbing power and holding on to it, often until death they do part.At a meeting with reporters, Boehner said, "My first job as speaker is to protect the institution."Really? Is that why Ohio voters sent him to Washington in 16 elections and his Republican colleagues elected and re-elected him speaker? Did he take an oath to preserve, protect and defend the institution of the House or the Constitution, which, if followed, offers protection enough?In 2010, I interviewed Boehner when he was minority leader and I asked him to cite the most important lesson he learned when Republicans lost their hard-won House majority in 2006. He replied, "Our team failed to live up to our own principles."Failing to live up to GOP principles, indeed, failing to articulate what those principles are, was largely the reason for the increase in conservative members who then dema

In [9]:
tokenizer

DistilBertTokenizerFast(name_or_path='/content/gdrive/MyDrive/FIMI_dataset/fake-news-classifier/', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
#{0: 'FAKE', 1: 'REAL'}
idx = 38

pipe = pipeline("text-classification", output_dir, tokenizer=tokenizer) #tokenizer=BERT_MODEL
# test the model
sample_title = test_df['text'][idx]
print(len(test_df['text'][idx]))
print(test_df['label'][idx])
# BERT is internally cutting everything after the 512 index in the training process
# We can play arround with using the first/ middle/ last 512 words or cut it in pices, classify all and see which class is chosen more often
# BigBERT might be able to deal with larger input
if len(sample_title) > 512:
    sample_title = sample_title[0:512]
print(len(sample_title))
pipe(sample_title, top_k=None)#[0]['score']

7918
true
512


[{'label': 'REAL', 'score': 0.9999072551727295},
 {'label': 'FAKE', 'score': 9.278723882744089e-05}]